### Configs

In [5]:
configs = {
    # Model Type, will support LR, SVC, RandomForest, and NB
    'model': 'lr',
    # Vectorizer Type (CV, TFIDF)
    'vectorizer': {'type': 'cv', 
                    'ngrams': (1,3),
                   'binary': False,
                   'stopwords': 'english',
                   'max_df_prop': 0.8,
                   'min_df_prop': 0.01
                  },
    'hyperparameters': {
        # Change these the most based on your model.
        "grid_search": True, # If true, your other hyperparemeters should be in a list.
        "C": [0.001, 0.01, 0.1, 1, 10],
        "solver": ["lbfgs"],
        "penalty": ["l2"],
        "max_iter": [100]
    }
}

configs

{'model': 'lr',
 'vectorizer': {'type': 'cv',
  'ngrams': (1, 3),
  'binary': False,
  'stopwords': 'english',
  'max_df_prop': 0.8,
  'min_df_prop': 0.01},
 'hyperparameters': {'grid_search': True,
  'C': [0.001, 0.01, 0.1, 1, 10],
  'solver': ['lbfgs'],
  'penalty': ['l2'],
  'max_iter': [100]}}

### Imports

In [1]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import re
import nltk

from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import plot_roc_curve, classification_report,

### Data Loading and Splitting

In [6]:
data = pd.read_csv('../data/combined_clean.csv', sep = "|")
print(data.shape)